In [1]:
from requests_html import HTML,HTMLSession
from bs4 import BeautifulSoup as bs
import datetime
import pandas as pd

In [2]:
import requests

def searchlinkedinjobs(start=0):
    params = {
        'keywords': '',
        'location': 'Kenya',
        'geoId': '100710459',
        'trk': 'public_jobs_jobs-search-bar_search-submit',
        'start' : start
    }
    response = requests.get('https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search', params=params)
    return response

response = searchlinkedinjobs()

In [ ]:
def external_link(job_link):
    resp = requests.get(job_link)
    soup = bs(resp.text, 'html.parser')
    apply_btn_link = soup.find(class_= 'sign-up-modal__direct-apply-on-company-site').find("a").get('href')
    apply_btn_link = requests.get(apply_btn_link).url
    return apply_btn_link

def jobdetails_info(job_api_details):
    resp = requests.get(job_api_details)
    jobdesc = bs(resp.text,'html.parser')
    job_desc_details = jobdesc.find(class_ = 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5')
    jobcriterialist = [{
        i.find(class_ = "description__job-criteria-subheader").text.strip():
        i.find(class_ = "description__job-criteria-subheader").find_next_sibling().text.strip()

    } for i in jobdesc.find("ul",class_ = "description__job-criteria-list").find_all("li")]

    out = {
        'jobcriterialist':jobcriterialist,
        'job_desc_details':job_desc_details
    }
    return out


def jobdetails(listings):
    try:
        job_title    = listings.find(class_="base-search-card__title").text.strip()
        company      = listings.find(class_="base-search-card__subtitle").text.strip()
        # location     = listings.find(class_ = 'job-search-card__location').text.strip()
        job_link     = listings.find(class_ ="base-card__full-link")
        if job_link:
            job_link = job_link.get("href")
            jobid = job_link.split("?")[0].split("-")[-1]
            job_api_details = f"https://ke.linkedin.com/jobs-guest/jobs/api/jobPosting/{jobid}"
            # get link to application
            print("Getting application details")
            application_link = external_link(job_link)
            print("Getting details")
            details = jobdetails_info(job_api_details)
        company_link = listings.find(class_ = "hidden-nested-link")
        if company_link:
            company_link = company_link.get('href')
        company_logo = listings.find("div",class_ = "search-entity-media").find("img").get("data-delayed-url")
        location     = listings.find(class_ = "job-search-card__location").text.strip()
        posting_time = listings.time.get('datetime')

        res = {
            "job_title"       :  job_title,
            "company"         :  company,
            "job_link"        :  job_link,
            "job_id"          :  jobid,
            "application_link":  application_link,
            "job_api_details" :  job_api_details,
            "company_logo"    :  company_logo,
            "location"        :  location,
            "posting_time"    :  posting_time,
            "crawl_time"      :  str(datetime.datetime.now()),
            "url"             :  response.url,
            "details"         :  details

        }
        return res
    except Exception as e:
        ...
    
start = 0
jd = []
while True:
    print("getting links")
    response = searchlinkedinjobs(start)
    soup = bs(response.text, 'html.parser').find_all("li")
    print("Parsing getting links")
    for i in range(len(soup)):
        jd.append(jobdetails(soup[i]))
    start = start + 25
    print('current start', start,end = "\r")
    if start >= 50:
        break

In [13]:
jd[1]['details']['job_desc_details']


<div class="show-more-less-html__markup show-more-less-html__markup--clamp-after-5">
<strong>Position title</strong> Cleaner IBO<p><br/></p><strong>Working location IBO Nairobi</strong><p><br/></p><strong> Region Eastern &amp; Southern Africa (ESAF</strong>)<p><br/></p><strong>Mission of the position </strong><p><br/></p>Cleaner is responsible for maintaining overal hygiene and cleanliness of all offices, toilets, boardrooms, storerooms and other facilities of the work place.<p><br/></p><strong>Main clients </strong><p><br/></p><ul><li>IB Coworkers.</li><li>Head of Mission.</li><li>Visitors.</li></ul><p><br/></p><strong>Key performance areas and main responsibilities </strong><p><br/></p><strong>Priority Task And Responsibilities</strong><p><br/></p><ul><li>Requests for and receives cleaning materials well in advance not to run short. Uses material economically (avoid misusing cleaning materials).</li><li>Requests for and receives office consumables and materials like coffee, tea, milk

In [ ]:
job_api_details = jd[0]['job_api_details']

# def jobdetails_info(job_api_details):
resp = requests.get(job_api_details)
jobdesc = bs(resp.text,'html.parser')
job_desc_details = jobdesc.find(class_ = 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5')
jobcriterialist = [{
    i.find(class_ = "description__job-criteria-subheader").text.strip():
    i.find(class_ = "description__job-criteria-subheader").find_next_sibling().text.strip()

} for i in jobdesc.find("ul",class_ = "description__job-criteria-list").find_all("li")]

# out = {
#     'jobcriterialist':jobcriterialist,
#     'job_desc_details':job_desc_details
# }
# return out
jobdesc.find("ul",class_ = "description__job-criteria-list").find_all("li")[0]
{key: value for (key, value) in (a.items() + b.items())}

In [308]:
4 

In [312]:
pd.DataFrame(jobcriterialist)

,Seniority level,Employment type,Job function,Industries
0,Entry level,NaN,NaN,NaN
1,NaN,Full-time,NaN,NaN
2,NaN,NaN,Administrative,NaN
3,NaN,NaN,NaN,International Affairs


In [264]:
apply_btn_link

'https://careers.un.org/lbw/jobdetail.aspx?id=194140'

In [216]:
import requests
from bs4 import BeautifulSoup

# Set the search query
query = "kcb bank kenya"

# Send an HTTP request to Google with the search query
html = requests.get(f"https://ke.linkedin.com/jobs/view/externalApply/3384864222?url=https%3A%2F%2Ftymeglobal%2Esnaphunt%2Ecom%2Fjob%2F7JVLN6HBMD-KE-103%3Fsource%3Dlinkedin&amp;urlHash=hEjr&amp;refId=GI%2FPjyzRUFvcHxi%2Fo962Gw%3D%3D&amp;trackingId=GK7EJH6OCnT9ajX7NGej3Q%3D%3D&amp;trk=public_jobs_apply-link-offsite_sign-up-modal-sign-up-later")




<Response [200]>

'https://tymeglobal.snaphunt.com/job/7JVLN6HBMD-KE-103?source=linkedin'

In [140]:
# soupsession = session.get(jd[1]['job_link'])
soupsession.html.xpath('//*[@id="topbar-apply"]')
# soupsession.get_attribute('href')
soupsession.html

<HTML url='https://ke.linkedin.com/jobs/view/warehouse-officer-at-l-or%C3%A9al-3402845830?refId=UinZ4fyWsD6AHWCUC%2BDDmg%3D%3D&trackingId=MYch%2FhsZIXpezg9WRap4fA%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card'>

In [195]:
url_ = "https://ke.linkedin.com/jobs/view/externalApply/3403724027?url=https%3A%2F%2Floa%2Eavature%2Enet%2Fcareers%2FJobDetail%2F140699&urlHash=3sbQ&refId=UinZ4fyWsD6AHWCUC%2BDDmg%3D%3D&trackingId=MYch%2FhsZIXpezg9WRap4fA%3D%3D&trk=public_jobs_apply-link-offsite_sign-up-modal-sign-up-later"
from urllib.parse import urlparse, parse_qs
print(url_)

https://ke.linkedin.com/jobs/view/externalApply/3403724027?url=https%3A%2F%2Floa%2Eavature%2Enet%2Fcareers%2FJobDetail%2F140699&urlHash=3sbQ&refId=UinZ4fyWsD6AHWCUC%2BDDmg%3D%3D&trackingId=MYch%2FhsZIXpezg9WRap4fA%3D%3D&trk=public_jobs_apply-link-offsite_sign-up-modal-sign-up-later


In [177]:
parsed_url = urlparse(url_)
# soupsession.request.query_params
parsed_url.query
url_data = parse_qs(parsed_url.query)
url_data

{'url': ['https://loa.avature.net/careers/JobDetail/140699'],
 'urlHash': ['3sbQ'],
 'refId': ['UinZ4fyWsD6AHWCUC+DDmg=='],
 'trackingId': ['MYch/hsZIXpezg9WRap4fA=='],
 'trk': ['public_jobs_apply-link-offsite_sign-up-modal-sign-up-later']}

In [182]:
parsed_url

ParseResult(scheme='https', netloc='ke.linkedin.com', path='/jobs/view/externalApply/3403724027', params='', query='url=https%3A%2F%2Floa%2Eavature%2Enet%2Fcareers%2FJobDetail%2F140699&urlHash=3sbQ&refId=UinZ4fyWsD6AHWCUC%2BDDmg%3D%3D&trackingId=MYch%2FhsZIXpezg9WRap4fA%3D%3D&trk=public_jobs_apply-link-offsite_sign-up-modal-sign-up-later', fragment='')

In [191]:


params = {'param1': 'value1', 'param2': 'value2'}

# Encode the parameters as a query string
# query_string = urlencode(params)
# query_string

scheme ='https'
netloc = 'ke.linkedin.com'
path = '/jobs/view/externalApply/3403724027'
params = {
    'url': 'https://loa.avature.net/careers/JobDetail/140699',
    'urlHash': '3sbQ',
    'refId': 'UinZ4fyWsD6AHWCUC+DDmg==',
    'trackingId': 'MYch/hsZIXpezg9WRap4fA==',
    'trk': 'public_jobs_apply-link-offsite_sign-up-modal-sign-up-later'
    }
main_link = {
    "scheme": scheme,
    "netloc": netloc,
    "path": path,
    "params": params,
    "query": urlencode(params),
    "fragment": ''
    }
url_ = urlunparse(main_link)
# url_
# print(url_)
url = f"{scheme}://{netloc}{path}?{urlencode(params)}"


In [196]:
job = session.get(url_)

In [197]:
job.url

'https://ke.linkedin.com/jobs/view/externalApply/3403724027?url=https%3A%2F%2Floa.avature.net%2Fcareers%2FJobDetail%2F140699&urlHash=3sbQ&refId=UinZ4fyWsD6AHWCUC%2BDDmg%3D%3D&trackingId=MYch%2FhsZIXpezg9WRap4fA%3D%3D&trk=public_jobs_apply-link-offsite_sign-up-modal-sign-up-later'